***Topic Modeling Using Latent Dirichlet Allocation (LDA)***

### installing packages

In [3]:
!pip install ipywidgets

In [6]:
pip install huggingface_hub[hf_xet]

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------- ----------------- 1.6/2.7 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install bertopic

  Using cached bertopic-0.17.0-py3-none-any.whl.metadata (23 kB)
  Using cached hdbscan-0.8.40-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached umap_learn-0.5.7-py3-none-any.whl.metadata (21 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
  Using cached numba-0.61.2-cp312-cp312-win_amd64.whl.metadata (2.9 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached bertopic-0.17.0-py3-none-any.whl (150 kB)
Using cached hdbscan-0.8.40-cp312-cp312-win_amd64.whl (726 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
Using cached umap_learn-0.5.7-py3-none-any.whl (88 kB)
Using cached huggingface_hub-0.31.2-py3-none-any.whl (484 kB)
Using cached numba-0.61.2-cp312-cp312-win_amd64.whl (2.8 MB)

In [2]:
!pip install gensim


   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ------ --------------------------------- 3.7/24.0 MB 19.8 MB/s eta 0:00:02
   ---------- ----------------------------- 6.0/24.0 MB 14.2 MB/s eta 0:00:02
   ----------------- ---------------------- 10.2/24.0 MB 16.8 MB/s eta 0:00:01
   ---------------------- ----------------- 13.6/24.0 MB 16.5 MB/s eta 0:00:01
   ---------------------------- ----------- 17.3/24.0 MB 17.0 MB/s eta 0:00:01
   ------------------------------------- -- 22.3/24.0 MB 17.8 MB/s eta 0:00:01
   ---------------------------------------  23.9/24.0 MB 17.8 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 15.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.9 MB ? eta -:--:--
   --- ------------------------------------ 3.9/45.9 MB 18.1 MB/s eta 0:00:03
   ------- -------------------------------- 8.1/45.9 MB 20.1 MB/s eta 0:00:02
   ------------ --------------------------- 13.9/45.9 MB 22.4 MB/s eta 0:0

  You can safely remove it manually.


In [6]:
nltk.download('punkt')       # For word_tokenize
nltk.download('wordnet')     # For lemmatizer
nltk.download('stopwords')   # For stopwords
nltk.download('omw-1.4')     # For lemmatizer support
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_percep

True

# Sample data 1

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.matutils import corpus2csc
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt

In [38]:
df=pd.read_csv('sample.csv')
sample_data=df['content']

In [39]:
# lemmatization and removing stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

#function to lemmatize and remove stopwords from the text data
def preprocess(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return words


#applying the function to the dataset
sample_data = sample_data.apply(preprocess)
sample_data

0     [found, subreddit, ,, hell, ., make, sad, angr...
1     [get, sub, gotten, lot, bigger, past, day, ’, ...
2     [lesbian, lucky, enough, live, way, year, ., a...
3     [saw, thought, great, ., thought, good, place,...
4     [tired, fed, seeing, men, uncle, culture, fyp,...
5     [4b, woman, always, considered, “, conventiona...
6     [men, fetishize, everything, ,, incest, ,, rap...
7     [certain, supplement, ,, especially, high, dos...
8     [hey, beautiful, soul, 💙, made, detox, social,...
9     [context, (, 18f, ), religious, conservative, ...
10                                              [found]
11    [,, sure, might, think, reaching, ,, wanted, s...
12    [dug, old, jewelry, making, kit, craft, room, ...
13    [decided, make, sub, woman, post, experience, ...
14    [*, apology, ,, *, *, mood, ., need, week, soc...
15    [news/information, resurfaced, regarding, 2006...
16    [good, book, recs, support, founding, principl...
17    [woman, living, uk, ,, stand, ., heart, br

The code below processes the data to remove words that appear in fewer than 5 documents and those that appear in more than 50% of the data

In [40]:
# Create a dictionary from the preprocessed data
dictionary = Dictionary(sample_data)

# Filter out words that appear in fewer than 5 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=5, no_above=0.5)

bow_corpus = [dictionary.doc2bow(text) for text in data]

# Train the LDA model
num_topics = 5
ldamodel = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=20, alpha='auto', eta='auto')

# Get the topics
topics = ldamodel.show_topics(num_topics=num_topics, num_words=10, log=False, formatted=False)

# Print the topics
for topic_id, topic in topics:
    print("Topic: {}".format(topic_id))
    print("Words: {}".format([word for word, _ in topic]))

Topic: 0
Words: [':', '?', "'s", 'men', 'woman', 'movement', 'think', 'know', 'http', "n't"]
Topic: 1
Words: ['’', 'want', 'woman', 'going', 'know', 'love', '(', ')', 'even', 'without']
Topic: 2
Words: ['’', 'woman', 'men', '“', '”', 'life', 'know', 'time', 'still', '4b']
Topic: 3
Words: ['people', "'s", 'get', 'right', 'made', 'would', "n't", '4b', 'support', 'also']
Topic: 4
Words: ['man', 'men', 'woman', 'want', 'also', '(', ')', 'make', 'think', "n't"]


# Full data 1

In [2]:
df2=pd.read_csv('merged.csv')
cleaned=df2[~df2['content'].isin(['[removed]', '[deleted]'])]
data2=cleaned['content'].astype(str)

In [36]:
data2

0        I cant be truly 4b because im in a relationshi...
1                                           I'm a lesbian.
2                          We need more people like you ❤️
3        I just ended things with a FWB who didn’t valu...
4                                                        🤏
                               ...                        
43639                                                  nan
43641                     They can’t even hide it anymore.
43642    I've been sober since 2016 and I have a lot of...
43643                                                  nan
43644    I'm so sick of stories like this yet another h...
Name: content, Length: 36839, dtype: object

In [17]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Helper to convert POS tag to WordNet POS
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

# Preprocessing function that returns a list
def preprocess(text):
    if pd.isnull(text):
        return []

    # Clean text
    text = re.sub(r'@\w+', '', text)         # Remove mentions
    text = re.sub(r'\d+', '', text)          # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip() # Normalize whitespace

    # Tokenize and lower
    tokens = word_tokenize(text.lower())

    # POS tagging
    pos_tags = pos_tag(tokens)

    # Lemmatize and remove stopwords/punctuation
    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos))
        for word, pos in pos_tags
        if word.isalpha() and word not in stop_words
    ]

    return lemmatized  # Return as a list

# Apply to your dataset
data3 = data2.apply(preprocess)

In [23]:
data3_cleaned = data3.dropna()
data3_cleaned = data3_cleaned[data3_cleaned.apply(lambda x: isinstance(x, list) and any(pd.notna(i) and i != '' for i in x))]
# Reset index
data = data3_cleaned.reset_index(drop=True)

In [24]:
data

0        [cant, truly, b, im, relationship, think, ive,...
1                                                [lesbian]
2                                     [need, people, like]
3        [end, thing, fwb, value, respect, tired, feel,...
4        [honestly, still, wide, variety, option, depen...
                               ...                        
36261                                                [nan]
36262                                [even, hide, anymore]
36263    [sober, since, lot, helpful, information, scie...
36264                                                [nan]
36265    [sick, story, like, yet, another, husband, kil...
Name: content, Length: 36266, dtype: object

In [25]:
# Create a dictionary from the preprocessed data
dictionary = Dictionary(data)

# Filter out words that appear in fewer than 5 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=5, no_above=0.5)

bow_corpus = [dictionary.doc2bow(text) for text in data]

# Train the LDA model
num_topics = 10
ldamodel = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=20, alpha='auto', eta='auto')

# Get the topics
topics = ldamodel.show_topics(num_topics=num_topics, num_words=10, log=False, formatted=False)

#Store and Print the topics
with open("LDA_topics.txt", "w", encoding="utf-8") as f:
    for topic_id, topic in topics:
        print("Topic: {}".format(topic_id))
        print("Words: {}".format([word for word, _ in topic]))
        

Topic: 0
Words: ['post', 'comment', 'share', 'thank', 'read', 'please', 'sub', 'write', 'book', 'add']
Topic: 1
Words: ['woman', 'b', 'movement', 'support', 'community', 'society', 'patriarchy', 'feminist', 'group', 'create']
Topic: 2
Words: ['na', 'block', 'gon', 'character', 'wan', 'revolution', 'trade', 'yep', 'privilege', 'wild']
Topic: 3
Words: ['buy', 'money', 'pay', 'class', 'business', 'product', 'art', 'car', 'sell', 'door']
Topic: 4
Words: ['rape', 'country', 'violence', 'state', 'birth', 'report', 'kill', 'domestic', 'law', 'risk']
Topic: 5
Words: ['child', 'kid', 'family', 'young', 'marry', 'girl', 'mother', 'husband', 'marriage', 'mom']
Topic: 6
Words: ['time', 'life', 'year', 'work', 'go', 'live', 'friend', 'find', 'start', 'day']
Topic: 7
Words: ['http', 'female', 'watch', 'video', 'social', 'medium', 'game', 'follow', 'online', 'content']
Topic: 8
Words: ['woman', 'men', 'get', 'want', 'even', 'would', 'one', 'know', 'u', 'need']
Topic: 9
Words: ['like', 'think', 'feel'

# BERT

## Full content

documentation: https://colab.research.google.com/drive/15zIjrFqZsJ-bbTueDOaXhhVfFIo-i22B?usp=sharing#scrollTo=FX_Ngb3PpRFG

In [27]:
from bertopic import BERTopic

In [8]:
df2=pd.read_csv('merged.csv')
cleaned=df2[~df2['content'].isin(['[removed]', '[deleted]'])]
data2=cleaned['content'].astype(str)
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(data2)

2025-05-20 12:47:21,008 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1152 [00:00<?, ?it/s]

2025-05-20 12:58:29,392 - BERTopic - Embedding - Completed ✓
2025-05-20 12:58:29,392 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-20 12:59:15,800 - BERTopic - Dimensionality - Completed ✓
2025-05-20 12:59:15,818 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-20 13:14:23,762 - BERTopic - Cluster - Completed ✓
2025-05-20 13:14:23,808 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-20 13:14:26,319 - BERTopic - Representation - Completed ✓


In [9]:
topic_model.get_topic_info().head(50)

# Topics of potential relevance

,Topic,Count,Name,Representation,Representative_Docs
0,-1,18004,-1_to_and_they_the,"[to, and, they, the, of, that, it, in, you, for]",[> You can ask a man nicely to do something an...
1,0,472,0_4b_movement_feminism_decentering,"[4b, movement, feminism, decentering, men, abo...","[Sorry, but 4b isn't a sort of revenge to make..."
2,1,358,1_korean_korea_south_movement,"[korean, korea, south, movement, 4b, feminists...",[and they say there is no 4b movement in Korea...
3,2,334,2_dad_she_mom_her,"[dad, she, mom, her, mother, father, my, he, h...",[My mom died of cancer in August this year. I ...
4,3,292,3_4b_movement_post_6b,"[4b, movement, post, 6b, 7b, relevant, join, i...","[4B for life! 🙌❤️, 4B, 4B!]"
5,4,290,4_nan_macaron_joseph_disastrous,"[nan, macaron, joseph, disastrous, himself, te...","[nan, nan, nan]"
6,5,252,5_christmas_holidays_thanksgiving_cook,"[christmas, holidays, thanksgiving, cook, cook...",[Nope. I'm spending a quiet Christmas by myse...
7,6,246,6_hugs_tk_oo_vcels,"[hugs, tk, oo, vcels, mah, darwin, omega, exho...","[Hugs💜, Hugs💜, Hugs💜]"
8,7,243,7_trans_cis_transphobia_transphobic,"[trans, cis, transphobia, transphobic, terf, t...","[Hot take, but I'd leave out trans men.\n\nTra..."
9,8,203,8_marriage_divorce_married_arranged,"[marriage, divorce, married, arranged, marry, ...","[I HATE happy marriage. What now?, Did she say..."


In [24]:
similar_topics, similarity = topic_model.find_topics("abortion", top_n=10); similar_topics

[49, 293, 69, 436, 419, 443, 163, 208, 355, 100]

In [26]:
# Look at top words for a given topic for search terms
topic_model.get_topic(436)

[('prolife', 0.05570381860933754),
 ('profamily', 0.05389564106865588),
 ('nutrition', 0.043165432415572254),
 ('pro', 0.04153836244622763),
 ('born', 0.041380523678211725),
 ('fetus', 0.03484536479205275),
 ('denying', 0.03351337136611297),
 ('prolifer', 0.02942240223083225),
 ('mitigates', 0.02942240223083225),
 ('resourcesslaves', 0.02942240223083225)]

In [27]:
from gensim.models import Word2Vec
import nltk
# nltk.download('punkt_tab')

# Preprocess the documents
tokenized_docs = []
for doc in data2:
  sentences = nltk.sent_tokenize(doc)
  for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    tokenized_docs.append(tokens)

# Train the Word2Vec model
model = Word2Vec(sentences=tokenized_docs, vector_size=100, window=5, min_count=1, workers=4)

In [40]:
## Now we have a model where each word has a similarity score to other words
## we can exploit this to search for new words relevant for our theme.

model.wv.most_similar("activism", topn=20)

[('outreach', 0.8726297616958618),
 ('providers', 0.856718897819519),
 ('medications', 0.8546890616416931),
 ('conditions', 0.8514281511306763),
 ('funding', 0.8509233593940735),
 ('separatism', 0.8503289818763733),
 ('korea', 0.8497620224952698),
 ('division', 0.8492761254310608),
 ('intervention', 0.8421908617019653),
 ('attitudes', 0.8421791195869446),
 ('definitions', 0.8419467806816101),
 ('deaths', 0.8418899774551392),
 ('acceptance', 0.8398092985153198),
 ('Peru', 0.8396855592727661),
 ('sustainability', 0.8391059041023254),
 ('russia', 0.8387611508369446),
 ('inequality', 0.838636040687561),
 ('excluding', 0.8383613228797913),
 ('fashion', 0.8381003141403198),
 ('resentment', 0.8376879096031189)]

In [ ]:
## We can also see which words are similar to a list of words

topic_4b_search_words = ['decenter','decentering','ignore','filter']
model.wv.most_similar(topic_4b_search_words, topn=10)

## Full clean data

In [5]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords, wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from bertopic import BERTopic

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.matutils import corpus2csc
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df=pd.read_csv('merged.csv')
df.head()

,username,user code,post code,parent id,title,content,time_posted,discussion_type,upvotes,downvotes,type,url,mod
0,covrtni,t2_i1uhmwvw,t1_ky8i5zm,t3_1bs8swh,NaN,I cant be truly 4b because im in a relationshi...,1.712354e+09,NaN,2,0,comment,/r/4bmovement/comments/1bs8swh/weekly_discussi...,False
1,Unable_Pineapple9211,t2_b10n6x0rk,t1_kyts0zn,t3_1bs8swh,NaN,I'm a lesbian.,1.712697e+09,NaN,2,0,comment,/r/4bmovement/comments/1bs8swh/weekly_discussi...,False
2,dahlia_74,t2_pblwlerm7,t1_kyydu93,t3_1bwr4cu,NaN,We need more people like you ❤️,1.712772e+09,NaN,1,0,comment,/r/4bmovement/comments/1bwr4cu/from_now_on_my_...,False
3,dahlia_74,t2_pblwlerm7,t1_kyyemc7,t3_1bs8swh,NaN,I just ended things with a FWB who didn’t valu...,1.712773e+09,NaN,5,0,comment,/r/4bmovement/comments/1bs8swh/weekly_discussi...,False
4,readditredditread,t2_1rh4e0ns,t1_kyzs2s9,t3_1bkrqs8,NaN,🤏,1.712789e+09,NaN,1,0,comment,/r/4bmovement/comments/1bkrqs8/the_4b_movement...,False


In [7]:
# POS tag to WordNet tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Text cleaning and preprocessing
def preprocess(text):
    if pd.isnull(text) or text.strip().lower() in ['removed', 'eliminated', '']:
        return np.nan

    text = re.sub(r'@\w+', '', text)         # Remove mentions
    text = re.sub(r'\d+', '', text)          # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip() # Normalize whitespace

    tokens = word_tokenize(text.lower())
    pos_tags = pos_tag(tokens)

    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos))
        for word, pos in pos_tags
        if word.isalpha() and word not in stop_words
    ]

    return ' '.join(lemmatized) if lemmatized else np.nan

# Step 1: Combine 'title' and 'content'
df['combined'] = df[['title', 'content']].fillna('').agg(' '.join, axis=1)

# Step 2: Clean text
df['cleaned'] = df['combined'].apply(preprocess)

# Step 3: Drop empty/invalid rows
cleaned_df = df['cleaned'].dropna().reset_index(drop=True)

NameError: name 'stop_words' is not defined

In [32]:
# Fit BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(cleaned_df)

2025-05-19 21:54:50,573 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1346 [00:00<?, ?it/s]

2025-05-19 22:02:27,682 - BERTopic - Embedding - Completed ✓
2025-05-19 22:02:27,698 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-19 22:04:07,727 - BERTopic - Dimensionality - Completed ✓
2025-05-19 22:04:07,751 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-19 22:40:42,821 - BERTopic - Cluster - Completed ✓
2025-05-19 22:40:43,464 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-19 22:40:46,976 - BERTopic - Representation - Completed ✓


In [33]:
# Topics of potential relevance
topics=topic_model.get_topic_info().head(30)

topics

,Topic,Count,Name,Representation,Representative_Docs
0,-1,19170,-1_woman_men_life_want,"[woman, men, life, want, like, get, make, thin...",[year old forb movement year without realize t...
1,0,3281,0_remove_awesomeeee_reductionary_tired,"[remove, awesomeeee, reductionary, tired, tmi,...","[remove, remove, remove]"
2,1,2369,1_remove_hahahahhahahah_hannam_pleeease,"[remove, hahahahhahahah, hannam, pleeease, orn...","[remove, remove, remove]"
3,2,784,2_vote_white_trump_conservative,"[vote, white, trump, conservative, liberal, el...","[dont vote man election country, yeah ya part ..."
4,3,383,3_abortion_pregnancy_pregnant_baby,"[abortion, pregnancy, pregnant, baby, birth, b...","[abortion ban good option, abortion woman preg..."
5,4,372,4_gun_spray_carry_defense,"[gun, spray, carry, defense, pepper, knife, we...","[buy gun learn use carry, woman martial art tr..."
6,5,319,5_rape_rapist_consent_rap,"[rape, rapist, consent, rap, assault, crime, v...",[woman support rapist rapist would never say m...
7,6,312,6_makeup_dress_wear_hair,"[makeup, dress, wear, hair, beauty, clothes, s...","[beauty makeup, go still wear makeup want also..."
8,7,304,7_movement_protest_radical_participate,"[movement, protest, radical, participate, femi...",[new maybe try read movement say world turn up...
9,8,297,8_iud_hysterectomy_uterus_period,"[iud, hysterectomy, uterus, period, hormonal, ...",[period almost year iud though awesome otherwi...


In [34]:
topics.to_csv('BERT_topics.csv', index=False)

In [1]:
topic_model.get_topic(0)

NameError: name 'topic_model' is not defined